# Etape 1 - Visualisation
Dans ce notebook, on traite la première partie de la méthode, qui consiste en la visualisation, la sélection et le formattage des données que l'on va utiliser par la suite.\
Le module présenté ici est un outil développé en Python pour visualiser les enregistrements d'EEG (électroencéphalogramme). Il se compose d'une classe nommée Visualisation et de plusieurs méthodes.

La méthode init prend trois arguments : patient, description, et D. Le paramètre patient est une chaîne de caractères qui contient le nom du patient pour lequel les données d'EEG seront visualisées. Le paramètre description est un DataFrame contenant des informations sur les enregistrements EEG. Enfin, le paramètre D est une durée spécifique d'enregistrement que l'utilisateur souhaite visualiser. Cette méthode charge les données d'EEG pour le patient spécifié, sélectionne les enregistrements EEG de durée D, et formate ces enregistrements pour une visualisation ultérieure.

La méthode chargement prend en entrée les paramètres patient et description et renvoie les chemins des fichiers de données ainsi que des informations spécifiques au patient.

La méthode selection prend en entrée un DataFrame (df) et une durée d'enregistrement (D) et renvoie une liste des noms d'enregistrements EEG à concaténer (name_enr_list) et une liste des noms d'enregistrements EEG qui seront utilisés pour nommer le fichier de sortie (name_fin_list).

La méthode lecture prend une liste de noms d'enregistrements et une liste de fichiers de données en entrée, et renvoie une liste de DataFrames pandas contenant les données d'EEG pour chaque enregistrement de la liste.

La méthode formattage prend en entrée une liste de DataFrames pandas, une liste de fichiers de données, et une DataFrame d'informations sur les enregistrements EEG. Cette méthode formate les DataFrames pour une visualisation ultérieure en ajoutant des informations sur le temps et les étiquettes.

La méthode concatenation prend en entrée une liste de DataFrames pandas formatés et une DataFrame d'informations sur les enregistrements EEG, et renvoie un DataFrame pandas qui contient toutes les données d'EEG concaténées et formatées pour la visualisation.

En résumé, ce module est un outil pratique pour visualiser les enregistrements d'EEG d'un patient spécifique, en sélectionnant des enregistrements d'une durée spécifique et en concaténant ces enregistrements pour une visualisation ultérieure. Cette méthode peut être utilisée dans le cadre de l'analyse des données d'EEG dans un contexte de recherche clinique ou de diagnostic médical.

In [1]:
# Modules
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from Visualisation import Visualisation
from Affichage import Affichage

In [2]:
# Paramètres généraux
patients = [f'chb{i:02d}' for i in range(1, 24)]
description = pd.read_csv('../../Data/description.csv', delimiter=';')
D = 120

In [9]:
# Chargement par patient
patient_nb = 1
patient = patients[patient_nb - 1]
visu = Visualisation(patient, description, D)

In [10]:
# Telechargement des csv concaténés
visu.telechargement(patient)

In [ ]:
# Téléchargement des images
aff = Affichage(patient)
aff.aff_all_eeg()